<a href="https://colab.research.google.com/github/SushovitNanda/SemEval-Food-Hazards/blob/main/PC_70.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
%%capture
!pip install transformers
!pip install datasets --upgrade
#!pip install torch
import os
import pandas as pd
!pip install evaluate
import evaluate
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification, RobertaForSequenceClassification, RobertaTokenizer
from transformers import DataCollatorWithPadding, EarlyStoppingCallback
from datasets import Dataset
import warnings
warnings.filterwarnings("ignore")

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"


In [2]:

!wget https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_train.csv
!wget https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_val.csv

--2024-12-04 12:31:47--  https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12866710 (12M) [text/plain]
Saving to: ‘incidents_train.csv’

incidents_train.csv 100%[===================>]  12.27M  --.-KB/s    in 0.04s   

2024-12-04 12:31:49 (326 MB/s) - ‘incidents_train.csv’ saved [12866710/12866710]

--2024-12-04 12:31:49--  https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_val.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sen

# Product Category

In [2]:
# Load the dataset
train = pd.read_csv('incidents_train.csv')

# Combine 'title' and 'text' columns to create input data
train['input_text'] = train['title'] + " " + train['text']

# Encode the target labels
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['product-category'])

# Train-test split using stratification
train_df, test_df = train_test_split(
    train[['input_text', 'label']],
    test_size=0.2,
    stratify=train['label'],
    random_state=42
)

# Convert the dataframes into Hugging Face's Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load the RoBERTa tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Define a tokenization function
def tokenize_function(example):
    return tokenizer(example['input_text'], truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set up a data collator to pad inputs dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load RoBERTa model for sequence classification
num_labels = len(label_encoder.classes_)
model = DistilBertForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english", num_labels=num_labels, ignore_mismatched_sizes=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
# Load the F1 metric and specify macro averaging
f1_metric = evaluate.load("f1")

# Define the compute_metrics function to maximize F1 macro average
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Compute the F1 macro average
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    return {"f1": f1["f1"]}

# Update training arguments to focus on F1 score
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Use F1 score as the metric for saving best model
    greater_is_better=True       # Ensure higher F1 is considered better
)

# Redefine the Trainer with the updated compute_metrics and early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Use F1 macro for evaluation
)

# Add EarlyStoppingCallback to the Trainer with patience of 4 epochs
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4))

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,1.240400,1.250854,0.357054
2,0.997500,1.015353,0.476702
3,0.467700,0.952652,0.513218
4,0.342700,1.003151,0.568702
5,0.187000,1.064147,0.567596
6,0.041400,1.210209,0.572187
7,0.108900,1.304214,0.592061
8,0.186700,1.396282,0.639499
9,0.025100,1.495071,0.649619


Epoch,Training Loss,Validation Loss,F1
1,1.240400,1.250854,0.357054
2,0.997500,1.015353,0.476702
3,0.467700,0.952652,0.513218
4,0.342700,1.003151,0.568702
5,0.187000,1.064147,0.567596
6,0.041400,1.210209,0.572187
7,0.108900,1.304214,0.592061
8,0.186700,1.396282,0.639499
9,0.025100,1.495071,0.649619
10,0.004500,1.548731,0.670105


TrainOutput(global_step=7635, training_loss=0.31958864121775604, metrics={'train_runtime': 3376.3062, 'train_samples_per_second': 60.199, 'train_steps_per_second': 7.538, 'total_flos': 8071132955225172.0, 'train_loss': 0.31958864121775604, 'epoch': 15.0})

In [4]:
# Evaluate the model
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

# Classification report
target_names = label_encoder.classes_
print(classification_report(test_df['label'], preds, target_names=target_names))

                                                   precision    recall  f1-score   support

                              alcoholic beverages       1.00      1.00      1.00        12
                      cereals and bakery products       0.81      0.75      0.78       134
     cocoa and cocoa preparations, coffee and tea       0.82      0.74      0.78        42
                                    confectionery       0.64      0.47      0.54        34
dietetic foods, food supplements, fortified foods       0.78      0.81      0.79        26
                                    fats and oils       0.67      0.50      0.57         4
                                   feed materials       0.00      0.00      0.00         1
                   food additives and flavourings       1.00      0.50      0.67         2
                           food contact materials       0.00      0.00      0.00         1
                            fruits and vegetables       0.77      0.80      0.79       10